In [3]:
import subprocess, os, csv, pandas as pd
import wget, urllib
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import Entrez
import Bio
from Bio import UniGene

from Bio.Alphabet import generic_dna

$\text{Searching bacteriophages in virus list and hosts Tax_ID list}$

In [4]:
# Here I'm searching bacteriophages in the table (from Virus-Host DB FTP) and saving  result in list
# Also I saving host names in anpther list
virus_host_info = []
host_name = []
host_id = []
with open("/home/emil/Desktop/Science/data/virushostdb/virushostdb.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t",)
    for line in tsvreader:
        if ('Bacteria' in line[9]):
            virus_host_info.append(line)
            host_name.append(line[8])
            host_id.append(line[7])
        if (line[9] == 'host lineage'):
            virus_host_info.append(line)

$\text{Searching hosts genome}$

In [6]:
data = pd.read_csv('/home/emil/Desktop/Science/data/virushostdb/assembly_summary_genbank.txt', sep="\n", header=None) #parsing
Assembly_summary_genbank = []
taxid = [] 
non_complete_genome_id = [] #Here IDs of organisms without full genome
link_to_sequence = [] #Here link to dowload data in FTP
for i in range(1,len(data[0])):
    s = data[0][i].split('\t')
    taxid.append(s[5])
    Assembly_summary_genbank.append(s)
for ID in host_id:
    if ID in taxid:
        index = taxid.index(ID)
        virus_index = host_id.index(ID) + 1
        if (Assembly_summary_genbank[index][13] == 'Full' or Assembly_summary_genbank[index][11] == 'Chromosome'):
            link_to_sequence.append(Assembly_summary_genbank[index][19])
        else:
            non_complete_genome_id.append(ID)
    else:
        non_complete_genome_id.append(ID)

$\text{Downloading Data}$

In [ ]:
unique_link = set(link_to_sequence)
unique_link = list(unique_link)
for i in range (len(link_to_sequence)):
    string = unique_link[i].split('/')
    link = unique_link[i] + '/' + string[len(string) - 1] + '_genomic.fna.gz'
    os.system('wget -r %s' %link)